In [ ]:
from pathlib import Path
from copy import deepcopy


import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import hessian
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

from topostats.filters import Filters
from topostats.io import LoadScans
from topostats.io import read_yaml

from topostats.plottingfuncs import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()

In [ ]:
def detect_ridges(gray, sigma=1.0):
    H_elems = hessian_matrix(gray, sigma=sigma, order="rc")
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    return maxima_ridges, minima_ridges

In [ ]:
# FILE_DIR = Path("./all_data")
# OUTPUT_DIR = Path("./training_data/images_edge_detection_lower_all_sigma_4")

FILE_DIR = Path("/Users/sylvi/topo_data/cats/eddie-haribo/on-target/data/")
OUTPUT_DIR = Path("/Users/sylvi/topo_data/cats/eddie-haribo/on-target/")
CONFIG_PATH = Path("/Users/sylvi/topo_data/cats/catsconf.yaml")


files = list(FILE_DIR.glob("*.spm"))
print(files)

# Load the data
loadscans = LoadScans(files, channel="Height")
loadscans.get_data()
topostats_data_dictionary = loadscans.img_dict

print(topostats_data_dictionary.keys())

filter_config = read_yaml(CONFIG_PATH)["filter"]
filter_config.pop("run")
filter_config["remove_scars"]["run"] = True
print(filter_config)

for index, (image_name, topostats_object) in enumerate(topostats_data_dictionary.items()):
    print(topostats_object.keys())
    image = topostats_object["image"]
    pixel_to_nm_scaling = topostats_object["px_2_nm"]
    filter_config_copy = deepcopy(filter_config)
    print(filter_config_copy)
    filters = Filters(image=image, filename=image_name, pixel_to_nm_scaling=pixel_to_nm_scaling, **filter_config_copy)
    filters.filter_image()
    flattened_image = filters.images["gaussian_filtered"]

    minima_edges, maxima_edges = detect_ridges(flattened_image, sigma=4.0)

    # plt.imsave(OUTPUT_DIR / f"training_image_{index}.png", minima_edges, cmap="binary")

    plt.imsave(OUTPUT_DIR / "flattened" / f"flattened_{index}.png", flattened_image)
    plt.imsave(OUTPUT_DIR / "ridges" / f"ridges_{index}.png", maxima_edges, cmap="binary")

    # fig, ax = plt.subplots(1, 3, figsize=(18, 36))
    # ax[0].imshow(flattened_image)
    # ax[0].set_title("flattened")
    # ax[1].imshow(minima_edges)
    # ax[1].set_title("minima edge detection")
    # ax[2].imshow(maxima_edges)
    # ax[2].set_title("maxima edge detection")
    # plt.show()

In [ ]:
# Load image and save it as npy and png

data_dir = Path("/Users/sylvi/topo_data/cats/training_data/cropped/cropped_labels/")
output_dir = Path("/Users/sylvi/topo_data/cats/training_data/cropped/cropped_labels/")

VMIN = -8
VMAX = 8

# Load the data
data = np.load(data_dir / "mask_75.npy")

plt.imshow(data, cmap=cmap, vmin=VMIN, vmax=VMAX)
plt.show()

# Save the image as png with the colormap and vmin and vmax
plt.imsave(output_dir / "alice_catenane.png", data, cmap=cmap, vmin=VMIN, vmax=VMAX)
# plot as npy
np.save(output_dir / "alice_catenane.npy", data)

In [ ]:
# Load topostats image, crop and save as npy and png

import h5py

data_dir = Path("/Users/sylvi/topo_data/cats/training_data/high_res_additions/output/processed/")
assert data_dir.exists()

OUTPUT_DIR = Path("/Users/sylvi/topo_data/cats/training_data/high_res_additions/cropped_molecules/")

files = list(data_dir.glob("*.topostats"))
# sort files
files = sorted(files)
for file in files:
    # print(file)
    pass

file_num = 32
file = files[file_num]
# Load the data
with h5py.File(file, "r") as f:
    # print keys
    # print(list(f.keys()))

    image = f["image"][:]

    print(f"image shape: {image.shape}")

    plt.imshow(image, cmap=cmap, vmin=-3, vmax=4)
    plt.show()

In [ ]:
# Crop the image

crop_size = 370
crop_x = 330
crop_y = 350
assert crop_x + crop_size < image.shape[0]
assert crop_y + crop_size < image.shape[1]

cropped_image = image[crop_y : crop_y + crop_size, crop_x : crop_x + crop_size]

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(cropped_image, cmap=cmap, vmin=-3, vmax=4)
plt.show()

# Get number of files in the output directory
output_files = list(OUTPUT_DIR.glob("*.npy"))
file_index = len(output_files)

In [ ]:
# Save the image as png with the colormap and vmin and vmax
OUTPUT_DIR.mkdir(exist_ok=True)
plt.imsave(OUTPUT_DIR / f"image_{file_index}.png", cropped_image, cmap=cmap, vmin=-3, vmax=4)
np.save(OUTPUT_DIR / f"image_{file_index}.npy", cropped_image)